# Jadwal 1 bulan awal waktu sholat

#### Package dan Fungsi yang dibutuhkan

In [1]:
import ephem #library untuk menghitung posisi benda langit(deklinasi, ra, dll) berdasarkan lokasi dan waktu tertentu.
import math as m #menyediakan fungsi matematika, ex:sin,cos,tan,radians,degrees,pi,dll
from math import pi #digunakan untuk konversi dari radian ke derajat.
from datetime import datetime, timedelta, timezone #untuk manipulasi tanggal dan waktu
from time import strftime #memformat waktu ke dalam tipe data string
import pandas as pd #Membuat array Menyimpan jadwal sholat dalam format tabel 

#### Fungsi Konversi Derajat-Desimal dan Desimal-Derajat

In [2]:
#Fungsi konversi derajat ke desimal
def deg_to_des(deg, menit, detik):
    sign    = -1 if deg < 0 else 1
    desimal = abs(deg)+(menit / 60) + (detik/3600)
    return desimal * sign
#Fungsi konversi desimal ke derajat
def format_dms(desimal):
    # Tentukan tanda positif/negatif untuk milai derajat
    sign = -1 if desimal < 0 else 1
    decimal_degrees = abs(desimal)
    # Hitung derajat
    degrees = int(decimal_degrees) * sign #mengambil nilai bil. bulan untuk derajat dari desimal_degrees Ex: 7.3347 = 7 derajat
    # Hitung menit
    sisa_DD = decimal_degrees - abs(degrees) #mengambil nilai desimal dari decimal_degrees Ex: 7.3347 = 0.3347
    total_minutes = sisa_DD * 60 #mengambil nilai menit desimal dari decimal_degrees Ex: 0.3347 = 20.082 menit
    minutes = int(total_minutes) #mengambil nilai menit DMS dari total_minutes Ex: 20.082 = 20 menit
    # Hitung detik dan tentukan pembulatan
    sisa_menit = (total_minutes - minutes) * 60 #pecahan menit 0.5 setara dengan 30 detik, maka disini pecahan menit * 60 Ex: 20.082 = 0.082 = 4.92 detik
    if sisa_menit >= 30:
        minutes += 1
    # Jika menit mencapai 60, tambahkan ke derajat
    if minutes == 60:
        degrees += 1 if degrees >= 0 else -1
        minutes = 0
    return f"{degrees}° {minutes}'"


#### Data Lokasi Pengamat

In [3]:
#data lintang tempat
def Lintang_tempat(der_lt,mnt_lt,dtk_lt):
    lt_tempat  = {"derajat": der_lt, "menit" : mnt_lt, "detik" : dtk_lt} #satuan derajat
    desimal_lt = deg_to_des(lt_tempat["derajat"], lt_tempat["menit"], lt_tempat["detik"]) #derajat ke desimal
    return desimal_lt
#data Bujur tempat
def Bujur_tempat(der_bj,mnt_bj,dtk_bj):
    bj_tempat  = {"derajat": der_bj, "menit" : mnt_bj, "detik" : dtk_bj} #satuan derajat
    desimal_bj = deg_to_des(bj_tempat["derajat"], bj_tempat["menit"], bj_tempat["detik"]) #derajat ke desimal
    return desimal_bj
#Fungsi pengkodisian waktu daerah 
def nilaiC (wd) :
    if wd == "WIB":
        return 105
    elif wd == "WITA":
        return 120
    elif wd == "WIT":
        return 135
    else:
        return 0
#Fungsi jumlah hari yang diinginkan dalam jadwal waktu sholat (tanggal awal) 
def Tanggal_awal1(Th1,Bn1,Hr1):
    waktu_awal      = {"Year": Th1, "Month": Bn1, "Day": Hr1}
    tanggal_Awal2   = datetime(waktu_awal["Year"], waktu_awal["Month"], waktu_awal["Day"])
    return tanggal_Awal2
#Fungsi jumlah hari yang diinginkan dalam jadwal waktu sholat (tanggal akhir) 
def Tanggal_akhir1(Th2,Bn2,Hr2):
    waktu_akhir     = {"Year": Th2, "Month": Bn2, "Day": Hr2}
    tanggal_akhir1  = datetime(waktu_akhir["Year"], waktu_akhir["Month"], waktu_akhir["Day"])
    return tanggal_akhir1

#Implementasi Fungsi
decimal_lt    = Lintang_tempat(-8,40,00)
decimal_bj    = Bujur_tempat(115,13,10)
ketinggian    = 18 #dalam satuan meter
Wd            = nilaiC("WITA") #waktu daerah
tanggal_awal  = Tanggal_awal1(2024,12,1)
tanggal_akhir = Tanggal_akhir1(2024,12,31)
ihtiyat       = deg_to_des(0,2,0)


#### data matahari

In [4]:
#fungsi untuk mengetahui nilai deklinasi
def deklinasi(lat, long, date, elev): 
    Bali         = ephem.Observer()
    Bali.lat     = str(lat)
    Bali.lon     = str(long)
    Bali.elev    = elev
    Bali.date    = date  
    sun          = ephem.Sun(Bali)
    deklinasi_dt = float(sun.dec) / pi * 180 
    return deklinasi_dt
#fungsi menghitung equation of time
def eot (date):
    jd  = ephem.julian_date(date)
    u   = (jd - 2451545)/36525
    l0  = 280.46607 + 36000.7698 * u
    t   = ((-1 * (1789 + 237 * u)) * m.sin(m.radians(l0)) - (7146 - 62 * u) \
        * m.cos(m.radians(l0)) + (9934 - 14 * u) * m.sin(m.radians(2 * l0)) - (29 + 5 * u) \
        * m.cos(m.radians(2 * l0)) + (74 + 10 * u) * m.sin(m.radians(3 * l0)) + (320 - 4 * u) \
        * m.cos(m.radians(3 * l0)) - 212 * m.sin(m.radians(4 * l0)))
    eot = t/1000
    return eot / 60 
#fungsi mencari nilai semi-diameter
def get_sun_semi_diameter (lat, lon, date):
    #set lokasi pengamatan
    observer      = ephem.Observer()
    observer.lat  = str(lat) #Latitude
    observer.lon  = str(lon) #Longitude
    observer.date = date 
    #Mendapatkan objek matahari
    sun           = ephem.Sun(observer)
    #Mendapatkan semi diameter
    sun_semi_diameter = sun.radius
    return sun_semi_diameter
#ini fungsi untu mendapatkan sudut waktu
def sudut_waktu (h0, dec, lt):
    t1 = m.sin(m.radians(h0))/ m.cos(m.radians(lt))/ m.cos(m.radians(dec)) - m.tan(m.radians(lt)) * m.tan(m.radians(dec))
    return m.degrees(m.acos(t1))/ 15



#### Implementasi Rumus Waktu Sholat ke Python

In [5]:
def jadwal_waktu_sholat (lt, bj, elev, date, beda_bj):
    dek     = deklinasi (lt, bj, date, elev)
    eot_1   = eot (date)
    sd      = get_sun_semi_diameter (lt, bj, date)
#perhitungan awal waktu dzuhur
    def dzuhur(bj, eot, beda_bj):
        return (12 - eot + (beda_bj - bj)/ 15) + ihtiyat
#perhitungan awal waktu ashar
    def ashar(lt, bj, dek, eot, beda_bj):
        zm = abs ( dek - lt)
        h1 = m.tan(m.radians(zm)) + 1
        h0 = m.degrees(m.atan(1/h1))
        t0 = sudut_waktu (h0, dek, lt)
        return (12 + t0 - eot + (beda_bj - bj) / 15) + ihtiyat
#perhitungan awal waktu magrib
    def maghrib(lt, bj, dek, eot, beda_bj, sd, ketinggian):
        ref = deg_to_des(0, 34 , 0)
        dip = deg_to_des(0, 1.76, 0)* m.sqrt(ketinggian)
        h0  = -1* (dip + ref + sd)
        t0  = sudut_waktu (h0, dek, lt)
        return (12 + t0 - eot + (beda_bj - bj) / 15) + ihtiyat
#perhitungan waktu isya
    def isya (lt, bj, dek, eot, beda_bj, sd, ketinggian):
        ref = deg_to_des(0, 3, 0)
        dip = deg_to_des (0, 1.76, 0) * m.sqrt (ketinggian)
        h0  = -1 * (17 - (dip + ref + sd))
        t0  = sudut_waktu (h0, dek, lt)
        return (12 + t0 - eot + (beda_bj - bj) / 15) + ihtiyat
#perhitungan waktu subuh
    def subuh (lt, bj, dek, eot, beda_bj, sd, ketinggian):
        ref = deg_to_des(0, 3, 0)
        dip = deg_to_des (0, 1.76, 0) * m.sqrt (ketinggian)
        h0  = -1 * (19 - (dip + ref + sd))
        t0  = -1 * (sudut_waktu (h0, dek, lt))
        return (12 + t0 - eot + (beda_bj - bj) / 15) + ihtiyat
#perhitungan thulu
    def thulu(lt, bj, dek, eot, beda_bj, sd, ketinggian):
        ref = deg_to_des(0, 34, 0)
        dip = deg_to_des(0, 1.76, 0)*m.sqrt(ketinggian)
        h0  = -1* (dip + ref + sd)  
        t0  = -1 * (sudut_waktu (h0, dek, lt))
        return (12 + t0 - eot + (beda_bj - bj) / 15) + ihtiyat
#perhitungan dhuha
    def dhuha(lt, bj, dek, eot, beda_bj):
        h0 = deg_to_des(4, 30, 0)
        t0 = -1 * (sudut_waktu (h0, dek, lt))
        return (12 + t0 - eot + (beda_bj - bj) / 15) + ihtiyat

#implementasi fungsi Jadwal waktu sholat
    dz  = dzuhur (bj, eot_1, beda_bj,)
    ash = ashar (lt, bj, dek, eot_1, beda_bj)
    mgb = maghrib (lt, bj, dek, eot_1, beda_bj, sd, elev) 
    isy = isya (lt, bj, dek, eot_1, beda_bj, sd, elev)  
    sbh = subuh (lt, bj, dek, eot_1, beda_bj, sd, elev)
    thl = thulu (lt, bj, dek, eot_1, beda_bj, sd, elev) 
    dh  = dhuha (lt, bj, dek, eot_1, beda_bj)

    return dz, ash, mgb, isy, sbh, thl, dh

#### Jadwal Waktu SHolat Satu Bulan

In [ ]:
#Membuat Looping dan Array Jadwal Waktu Sholat
#Parameter tanggal dan fungsi Jadwal waktu sholat
#masukin data untuk tabel  #[] list {} Diconary () tupple
current_date = tanggal_awal 
data_jadwal = {"Tanggal" : [],
               "Dzuhur" : [],
               "Ashar" : [],
               "Maghrib" : [],
               "Isya" : [],
               "Subuh" : [],
               "Thulu" : [],
               "Dhuha" : []} 
while current_date <= tanggal_akhir:
    dz, ash, mgb, isy, sbh, thl, dh = jadwal_waktu_sholat (decimal_lt, decimal_bj, ketinggian, current_date, Wd)

    data_jadwal ["Tanggal"].append(f"{current_date.strftime('%Y/%m/%d')}")
    data_jadwal ["Dzuhur"].append(f"{format_dms(dz)}")
    data_jadwal ["Ashar"].append(f"{format_dms(ash)}")
    data_jadwal ["Maghrib"].append(f"{format_dms(mgb)}")
    data_jadwal ["Isya"].append(f"{format_dms(isy)}")
    data_jadwal ["Subuh"].append(f"{format_dms(sbh)}")
    data_jadwal ["Thulu"].append(f"{format_dms(thl)}")
    data_jadwal ["Dhuha"].append(f"{format_dms(dh)}")

    current_date += timedelta(days= 1)

pd.DataFrame(data_jadwal)


,Tanggal,Dzuhur,Ashar,Maghrib,Isya,Subuh,Thulu,Dhuha
0,2024/12/01,12° 10',15° 35',18° 27',19° 39',4° 33',5° 53',6° 16'
1,2024/12/02,12° 11',15° 36',18° 28',19° 39',4° 33',5° 53',6° 16'
2,2024/12/03,12° 11',15° 37',18° 28',19° 40',4° 33',5° 54',6° 16'
3,2024/12/04,12° 11',15° 37',18° 29',19° 40',4° 33',5° 54',6° 17'
4,2024/12/05,12° 12',15° 38',18° 29',19° 41',4° 33',5° 54',6° 17'
5,2024/12/06,12° 12',15° 38',18° 30',19° 42',4° 34',5° 55',6° 17'
6,2024/12/07,12° 13',15° 39',18° 30',19° 42',4° 34',5° 55',6° 18'
7,2024/12/08,12° 13',15° 39',18° 31',19° 43',4° 34',5° 55',6° 18'
8,2024/12/09,12° 13',15° 40',18° 31',19° 43',4° 35',5° 56',6° 18'
9,2024/12/10,12° 14',15° 41',18° 32',19° 44',4° 35',5° 56',6° 19'


#### Import tabel dataframe ke format excel

In [7]:
df = pd.DataFrame(data_jadwal)
df.to_excel("jadwal waktu sholat 1 bulan.xlsx")